In [14]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pathlib import Path

In [ ]:
activities = ['idle', 'running', 'stairs', 'walking']
data_collections = []

for activity in activities:
    activity_path = Path('data') / activity
    csv_files = list(activity_path.glob('*.csv'))
    if not csv_files:
        print(f"Не знайдено CSV файлів для активності: {activity}")
        continue
    print(f"Завантаження {len(csv_files)} файлів для активності: {activity}")
    df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
    df['activity'] = activity
    data_collections.append(df)

if data_collections:
    data = pd.concat(data_collections, ignore_index=True)
    print("Об'єднані дані:")
    print(data.head())
else:
    print("Не знайдено жодних даних для об'єднання.")
    exit()  

data['target'] = data['activity'].astype('category').cat.codes
print("Приклад даних з метками активності:")
print(data.head())

window_size = 10
stride = 5

def extract_features(data, window_size, stride):
    features = []
    labels = []

    for i in range(0, len(data) - window_size, stride):
        window = data.iloc[i:i + window_size]
        feature_dict = {}

        if len(window) < window_size:
            continue

        for column in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
            feature_dict[f'{column}_mean'] = window[column].mean()
            feature_dict[f'{column}_std'] = window[column].std()
            feature_dict[f'{column}_max'] = window[column].max()
            feature_dict[f'{column}_min'] = window[column].min()

        labels.append(window['target'].mode()[0])
        features.append(feature_dict)

    if not features:
        print("Нет сгенерированных признаков. Проверьте параметры window_size и stride.")
    else:
        print(f"Количество сгенерированных признаков: {len(features)}")

    return pd.DataFrame(features), np.array(labels)

X, y = extract_features(data, window_size, stride)
print("Пример сгенерированных признаков:")
print(X.head())

if not X.empty:
    print("Распределение классов в данных:", np.bincount(y))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Распределение классов в обучающей выборке:", np.bincount(y_train))

    svm_model = SVC(kernel='rbf')
    svm_model.fit(X_train, y_train)

    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    y_pred_svm = svm_model.predict(X_test)
    print("SVM Classification Report:")
    print(classification_report(y_test, y_pred_svm, zero_division=0))

    y_pred_rf = rf_model.predict(X_test)
    print("Random Forest Classification Report:")
    print(classification_report(y_test, y_pred_rf, zero_division=0))
else:
    print("Ошибка: сгенерированный DataFrame признаков пуст.")

Завантаження 1039 файлів для активності: idle
Завантаження 3408 файлів для активності: running
Завантаження 165 файлів для активності: stairs
Завантаження 1850 файлів для активності: walking
Об'єднані дані:
   accelerometer_X  accelerometer_Y  accelerometer_Z activity
0         1.000776         4.616021         8.576031     idle
1         0.718261         4.209007         8.446744     idle
2        -0.909797        -0.282516         9.203311     idle
3         5.099650         0.148441         8.418014     idle
4         1.762132        -0.162806         9.251195     idle
Приклад даних з метками активності:
   accelerometer_X  accelerometer_Y  accelerometer_Z activity  target
0         1.000776         4.616021         8.576031     idle       0
1         0.718261         4.209007         8.446744     idle       0
2        -0.909797        -0.282516         9.203311     idle       0
3         5.099650         0.148441         8.418014     idle       0
4         1.762132        -0.162806